### Import Libraries

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import os
import itertools
import math
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import random
import gurobipy as gp
from gurobipy import GRB
import tensorflow as tf
from tensorflow.keras import initializers
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
import yaml
from pathlib import Path

### Import modules

In [ ]:
import sys
sys.path.append("../src")

import data_exploration as de
import model_training as mt

### Load necessary directories

In [ ]:
current_dir = Path(os.getcwd())
root_dir = current_dir
while 'Portfolio Optimization using SPO' in root_dir.parts:
    root_dir = root_dir.parent
    if root_dir == Path(root_dir.root):
        print("Root directory not found.")
        break

In [ ]:
config_path = root_dir / "Portfolio Optimization using SPO" / "config" / "config.yml"
complete_data_path = root_dir / "Portfolio Optimization using SPO" / "data" / "dat_518_companies.csv"
toy_data_path = root_dir / "Portfolio Optimization using SPO" / "data" / "AAPL_toy.csv"
toy_cost_mat_path = root_dir / "Portfolio Optimization using SPO" / "data" / "cost_vec_toy.csv"

In [ ]:
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

### Import data

In [ ]:
# import data
df_toy_original = pd.read_csv(toy_data_path)
df_toy_final_returns = pd.read_csv(toy_cost_mat_path)

### Calculate covariance matrix ($\Sigma$) and risk factor ($\gamma$)

In [ ]:
# covatiance matrix calculation
df_toy_returns_transpose = np.transpose(df_toy_final_returns)
toy_sigma = np.cov(df_toy_returns_transpose)
toy_sigma_df = pd.DataFrame(toy_sigma)
toy_sigma_df

In [ ]:
# risk factor calculation
toy_avg_vec = np.ones(2)/2
toy_gamma = np.transpose(toy_avg_vec)@toy_sigma@toy_avg_vec
toy_gamma = 2.25*toy_gamma
print(toy_gamma)

### Initialize the model

In [ ]:
toy_n_rows, toy_n_cols = df_toy_original.shape
toy_n_feats = toy_n_cols-1

# Instantiate the model
model_toy_data = mt.get_model(n_feats = toy_n_feats)
model_toy_data.summary()

### Split data into train and test

In [ ]:
# training dataframe
df_toy_original_train, df_toy_original_test = train_test_split(df_toy_original, test_size=0.2, 
                                                               random_state=42, shuffle=False)
# cost vector
df_toy_final_returns_train, df_toy_final_returns_test = train_test_split(df_toy_final_returns, test_size=0.2, 
                                                                         random_state=42, shuffle=False)

In [ ]:
%%time
# Grid Search to find best hyper-parameters

# Parameters
grid_search_params = config["toy_grid_params"]

# Searching Best Parameters
results, best_params, error = mt.grid_search(df_toy_original, df_toy_final_returns, sigma=toy_sigma, gamma=toy_gamma, n_epoch=20, GridSearchParams=grid_search_params)

# Print results
print("Results:")
for res in results:
    print(res)

print("Best Parameters:", best_params)
print("Error:", error)

In [ ]:
# Train the model with best hyper-parameters
trained_toy_model, epoch_toy_loss_list = mt.SGD_regressor(df_toy_original, model_toy_data, df_toy_final_returns, toy_sigma, toy_gamma, learning_rate= 0.001, decay_rate=1.02, n_epochs=20, batch_size = 4)

### Plot loss progression with every epoch

In [ ]:
fig_toy = px.line(epoch_toy_loss_list).update_layout(title="Training Loss progression", xaxis_title="epochs", yaxis_title="SPO+ loss",legend={
            "title": "Loss Value"})
fig_toy.show()

### Testing the model on test data

In [ ]:
y_pred_toy = trained_toy_model(df_toy_original_test.iloc[:,0:toy_n_feats].values)
toy_spo_test_loss = mt.get_SPO_plus_testing_loss(df_toy_original, df_toy_final_returns_test, 
                                                 y_pred_toy, sigma=toy_sigma, gamma=toy_gamma)

print(f'The SPO+ loss on testing data for toy problem is {toy_spo_test_loss}')

In [ ]:
# Save the entire model as a `.keras` zip archive.
model_save_path = root_dir / "Portfolio Optimization using SPO" / "models" / "trained_toy_model.keras"
trained_toy_model.compile()
trained_toy_model.save(model_save_path, save_format='tf')

To load the model and test use below command

```
new_model = tf.keras.models.load_model(model_save_path)
```